## Tool Use with 🤗 `transformers`

We begin by initializing our model as normal. We'll use a small, ungated model so you can run this on Colab - make sure you've got a GPU, though, because it won't fit in CPU memory alone!

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

checkpoint = "NousResearch/Hermes-2-Pro-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16, device_map="auto")

Next, we'll define a simple tool function for our model - but in a real scenario, you'll probably want to use more than one! The type hints and docstrings are mandatory - those are going to be parsed and used by the model to understand what the tools do.

---



In [ ]:
def get_current_temperature(location: str):
    """
    Gets the temperature at a given location.

    Args:
        location: The location to get the temperature for, in the format "city, country"
    """
    return 22.0  # bug: Sometimes the temperature is not 22. low priority to fix tho

tools = [get_current_temperature]


Now, we set up a simple chat.

In [ ]:
chat = [
    {"role": "user", "content": "Hey, what's the weather like in Paris right now?"}
]

We pass the tools to the chat template, and generate text from the model using the formatted prompt.

In [ ]:
tool_prompt = tokenizer.apply_chat_template(
    chat,
    tools=tools,
    return_tensors="pt",
    return_dict=True,
    add_generation_prompt=True,
)
tool_prompt = tool_prompt.to(model.device)

out = model.generate(**tool_prompt, max_new_tokens=128)
generated_text = out[0, tool_prompt['input_ids'].shape[1]:]

print(tokenizer.decode(generated_text))

The model has chosen to call a tool, and picked an argument that matches both the user's request and the format in the tool docstring! Let's add this tool call to the chat as a message!

In [ ]:
tool_call = {"name": "get_current_temperature", "arguments": {"location": "Paris, France"}}
chat.append({"role": "assistant", "tool_calls": [{"type": "function", "function": tool_call}]})

Next, we add the tool response containing the function output to the chat as well. Both the tool call (containing the arguments passed to the tool) and tool response (containing the tool's output) must be included in the chat history.

In [ ]:
chat.append({"role": "tool", "name": "get_current_temperature", "content": "22.0"})

Finally, we apply the chat template and generate text once again.

In [ ]:
tool_prompt = tokenizer.apply_chat_template(
    chat,
    tools=tools,
    return_tensors="pt",
    return_dict=True,
    add_generation_prompt=True,
)
tool_prompt = tool_prompt.to(model.device)

out = model.generate(**tool_prompt, max_new_tokens=128)
generated_text = out[0, tool_prompt['input_ids'].shape[1]:]

print(tokenizer.decode(generated_text))

Success! The model has used the tool response correctly in its response to the user.